## Extract Poses from Amass Dataset

In [11]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
%matplotlib inline

import sys, os
import torch
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm
import csv


from human_body_prior.tools.omni_tools import copy2cpu as c2c

os.environ['PYOPENGL_PLATFORM'] = 'egl'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Please remember to download the following subdataset from AMASS website: https://amass.is.tue.mpg.de/download.php. Note only download the <u>SMPL+H G</u> data.
* ACCD (ACCD)
* HDM05 (MPI_HDM05)
* TCDHands (TCD_handMocap)
* SFU (SFU)
* BMLmovi (BMLmovi)
* CMU (CMU)
* Mosh (MPI_mosh)
* EKUT (EKUT)
* KIT  (KIT)
* Eyes_Janpan_Dataset (Eyes_Janpan_Dataset)
* BMLhandball (BMLhandball)
* Transitions (Transitions_mocap)
* PosePrior (MPI_Limits)
* HumanEva (HumanEva)
* SSM (SSM_synced)
* DFaust (DFaust_67)
* TotalCapture (TotalCapture)
* BMLrub (BioMotionLab_NTroje)

### Unzip all datasets. In the bracket we give the name of the unzipped file folder. Please correct yours to the given names if they are not the same.

### Place all files under the directory **./amass_data/**. The directory structure shoud look like the following:  
./amass_data/  
./amass_data/ACCAD/  
./amass_data/BioMotionLab_NTroje/  
./amass_data/BMLhandball/  
./amass_data/BMLmovi/   
./amass_data/CMU/  
./amass_data/DFaust_67/  
./amass_data/EKUT/  
./amass_data/Eyes_Japan_Dataset/  
./amass_data/HumanEva/  
./amass_data/KIT/  
./amass_data/MPI_HDM05/  
./amass_data/MPI_Limits/  
./amass_data/MPI_mosh/  
./amass_data/SFU/  
./amass_data/SSM_synced/  
./amass_data/TCD_handMocap/  
./amass_data/TotalCapture/  
./amass_data/Transitions_mocap/  

**Please make sure the file path are correct, otherwise it can not succeed.**

In [12]:
# Choose the device to run the body model on.
comp_device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [13]:
from human_body_prior.body_model.body_model import BodyModel

male_bm_path = './body_models/smplh/male/model.npz'
male_dmpl_path = './body_models/dmpls/male/model.npz'

female_bm_path = './body_models/smplh/female/model.npz'
female_dmpl_path = './body_models/dmpls/female/model.npz'

infant_bm_path = './body_models/smplh/infant/model.npz'
infant_dmpl_path = './body_models/dmpls/infant/model.npz'

num_betas = 10 # number of body parameters
num_dmpls = 0 # number of DMPL parameters

num_infant_betas = 8 # number of body parameters
num_infant_dmpls = 0 # number of DMPL parameters

male_bm = BodyModel(bm_fname=male_bm_path, num_betas=num_betas, num_dmpls=num_dmpls, dmpl_fname=male_dmpl_path).to(comp_device)
faces = c2c(male_bm.f)

female_bm = BodyModel(bm_fname=female_bm_path, num_betas=num_betas, num_dmpls=num_dmpls, dmpl_fname=female_dmpl_path).to(comp_device)
infant_bm = BodyModel(bm_fname=infant_bm_path, num_betas=num_infant_betas).to(comp_device) ## run without dmpls because they aren't available

In [14]:
import os
import shutil

data_folder='/workspaces/HumanML3D/infant_poses'
save_folder='/workspaces/HumanML3D/all_infant_poses' 

# Get all .pkl files in the specified folder
files = [f for f in os.listdir(data_folder) if f.endswith('.pkl')]

In [15]:
def get_fname(file):

    if "-" in file.split("_")[0]:
        fname = f'{file.split("_")[0].split("-")[1]}_{file.split("_")[1]}_{file.split("_")[2]}'
    else:
        fname = f'{file.split("_")[1]}_{file.split("_")[2]}_{file.split("_")[3]}'

    return fname

In [16]:
# Loop through the files and organize them into subfolders
for file in files:

    prefix = get_fname(file)
    
    # Create the subfolder path
    subfolder_path = os.path.join(save_folder, prefix)
    #print(subfolder_path)
    
    # Create the subfolder if it doesn't exist
    if not os.path.exists(subfolder_path):
        os.makedirs(subfolder_path)
    
    # Move the file into the corresponding subfolder
    #print(os.path.join(data_folder, file), subfolder_path)

    if not os.path.exists(os.path.join(subfolder_path, file)):
        shutil.copy(os.path.join(data_folder, file), subfolder_path)

print("Files have been organized into subfolders.")


Files have been organized into subfolders.


In [17]:
import pickle as pkl

def get_poses(folder):
    poses, trans, betas = [], [],[]

    for pkl_file in os.listdir(folder):
        with open(os.path.join(folder, pkl_file), 'rb') as f:

            pkl_poses = pkl.load(f)

            betas.append(list(pkl_poses['betas'][0]))
            poses.append(list(pkl_poses['body_pose'][0]))
            trans.append(list(pkl_poses['camera_translation'][0]))

    new_dict = {'betas':betas,
        'trans':trans,
        'poses':poses,
        'mocap_framerate':int(15),
        'gender':'infant'
        }
    
    return new_dict

In [53]:
#TODO: Change the path to the folder containing the subfolders
# get all poses from the subfolders

file_details = []
file_id = 0

for subfolder in os.listdir(save_folder):
    pkl_path = os.path.join(save_folder, subfolder)
    try:       
        all_poses = get_poses(pkl_path)

        # Check if 'betas', 'trans', or 'poses' fields are empty
        if not (all_poses['betas'] and all_poses['trans'] and all_poses['poses']):
            print(f"Skipping saving {subfolder} as 'betas', 'trans', or 'poses' are empty")
        
        else:
            out_dir = f'/workspaces/HumanML3D/amass_data/Infant/{subfolder}'
            out_file = f'infant_{subfolder}.npz'

            if not os.path.exists(out_dir):
                os.makedirs(out_dir)

            #source_path = f'./all_infant_poses/CHOP/{gma_id}/infant_{gma_id}.npy'  # Replace with your source path
            source_path = f'./pose_data/Infant/{subfolder}/infant_{subfolder}.npy'
            new_name = f'{str(file_id).zfill(5)}.npy'  # Replace with your new name
            start_frame = 0
            end_frame = len(all_poses['poses'])

            # Store the details for the CSV file
            file_details.append([source_path, start_frame, end_frame, new_name])
            file_id += 1
            
            np.savez(os.path.join(out_dir,out_file), **all_poses)
            #print(f'saving poses for {subfolder}')
    except:
        continue
        print(f'no good poses for {subfolder}')

# Write to CSV file after the loop
with open('index_infant.csv', mode='w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['source_path', 'start_frame', 'end_frame', 'new_name'])
    for details in file_details:
        csv_writer.writerow(details)

print("CSV file saved successfully")

CSV file saved successfully


In [20]:
paths = []
folders = []
dataset_names = []
for root, dirs, files in os.walk('./amass_data'):
#     print(root, dirs, files)
#     for folder in dirs:
#         folders.append(os.path.join(root, folder))
    folders.append(root)
    for name in files:
        dataset_name = root.split('/')[2]
        if dataset_name not in dataset_names:
            dataset_names.append(dataset_name)
        paths.append(os.path.join(root, name))

In [21]:
save_root = './pose_data'
save_folders = [folder.replace('./amass_data', './pose_data') for folder in folders]
for folder in save_folders:
    os.makedirs(folder, exist_ok=True)
group_path = [[path for path in paths if name in path] for name in dataset_names]

In [43]:
trans_matrix = np.array([[1.0, 0.0, 0.0],
                            [0.0, 0.0, 1.0],
                            [0.0, 1.0, 0.0]])
ex_fps = 15
def amass_to_pose(src_path, save_path):
    bdata = np.load(src_path, allow_pickle=True)
    fps = 0
    try:
        fps = bdata['mocap_framerate']
        frame_number = bdata['trans'].shape[0]
    except:
#         print(list(bdata.keys()))
        return fps
    
    fId = 0 # frame id of the mocap sequence
    pose_seq = []
    
    if bdata['gender'] == 'male':
        bm = male_bm
    if bdata['gender'] == 'female':
        bm = female_bm
    if bdata['gender'] == 'infant':
        bm = infant_bm
    
    down_sample = int(fps / ex_fps)
#     print(frame_number)
#     print(fps)
    
    with torch.no_grad():
        for fId in range(0, frame_number, down_sample):
            root_orient = torch.Tensor(bdata['poses'][fId:fId+1, :3]).to(comp_device) # controls the global root orientation
            pose_body = torch.Tensor(bdata['poses'][fId:fId+1, 3:66]).to(comp_device) # controls the body
            pose_hand = torch.Tensor(bdata['poses'][fId:fId+1, 66:]).to(comp_device) # controls the finger articulation
            betas = torch.Tensor(bdata['betas'][:num_infant_betas][np.newaxis]).to(comp_device) # controls the body shape
            trans = torch.Tensor(bdata['trans'][fId:fId+1]).to(comp_device)    
            body=bm(body_pose=pose_body)
            #body = bm(pose_body=pose_body, pose_hand=None, betas=betas, root_orient=root_orient)
            joint_loc = body.Jtr[0] + trans
            pose_seq.append(joint_loc.unsqueeze(0))
    
    pose_seq = torch.cat(pose_seq, dim=0)
    
    pose_seq_np = pose_seq.detach().cpu().numpy()
    pose_seq_np_n = np.dot(pose_seq_np, trans_matrix)
    
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    np.save(save_path, pose_seq_np_n)
    
    return fps

In [23]:
group_path = group_path
all_count = sum([len(paths) for paths in group_path])
cur_count = 0

In [44]:
import time

for paths in group_path:
    dataset_name = paths[0].split('/')[2]

    if dataset_name == 'Infant':
        print(dataset_name)
        pbar = tqdm(paths)
        pbar.set_description('Processing: %s'%dataset_name)
        fps = 15
        for path in pbar:
            save_path = path.replace('./amass_data', './pose_data')

            save_path = save_path[:-3] + 'npy'
            print(path)

            if os.path.isfile(save_path):
                print(f'file exists, moving on')
                continue
            else:
                fps = amass_to_pose(path, save_path)
            
        cur_count += len(paths)
        print('Processed / All (fps %d): %d/%d'% (fps, cur_count, all_count) )
        time.sleep(0.5)
    else:
        print(f'Skipping {dataset_name}. Only processing Infant data for now')

Skipping Transitions_mocap. Only processing Infant data for now
Skipping DFaust_67. Only processing Infant data for now
Skipping TotalCapture. Only processing Infant data for now
Skipping Eyes_Japan_Dataset. Only processing Infant data for now
Skipping BMLhandball. Only processing Infant data for now
Skipping TCD_handMocap. Only processing Infant data for now
Skipping SFU. Only processing Infant data for now
Infant


Processing: Infant:   0%|          | 0/13 [00:00<?, ?it/s]

./amass_data/Infant/19_2_1/infant_19_2_1.npz


Processing: Infant:   8%|▊         | 1/13 [00:02<00:34,  2.91s/it]

./amass_data/Infant/26_1_1/infant_26_1_1.npz


...

./amass_data/Infant/11_1_2/infant_11_1_2.npz


Processing: Infant:  92%|█████████▏| 12/13 [00:42<00:03,  3.01s/it]

./amass_data/Infant/23_1_1/infant_23_1_1.npz


Processing: Infant: 100%|██████████| 13/13 [00:44<00:00,  3.44s/it]


Processed / All (fps 15): 13/14068
Skipping EKUT. Only processing Infant data for now
Skipping KIT. Only processing Infant data for now
Skipping SSM_synced. Only processing Infant data for now
Skipping MPI_mosh. Only processing Infant data for now
Skipping BMLmovi. Only processing Infant data for now
Skipping BioMotionLab_NTroje. Only processing Infant data for now
Skipping MPI_Limits. Only processing Infant data for now
Skipping CMU. Only processing Infant data for now
Skipping HumanEva. Only processing Infant data for now
Skipping ACCAD. Only processing Infant data for now
Skipping MPI_HDM05. Only processing Infant data for now


This will take a few hours for all datasets, here we take one dataset as an example

To accelerate the process, you could run multiple scripts like this at one time.

In [10]:
import time
for paths in group_path:
    dataset_name = paths[0].split('/')[2]
    pbar = tqdm(paths)
    pbar.set_description('Processing: %s'%dataset_name)
    fps = 0
    for path in pbar:
        save_path = path.replace('./amass_data', './pose_data')
        save_path = save_path[:-3] + 'npy'
        fps = amass_to_pose(path, save_path)
        
    cur_count += len(paths)
    print('Processed / All (fps %d): %d/%d'% (fps, cur_count, all_count) )
    time.sleep(0.5)

Processing: Transitions_mocap: 100%|██████████| 110/110 [00:01<00:00, 62.55it/s]


Processed / All (fps 120): 110/14055


Processing: DFaust_67: 100%|██████████| 139/139 [00:01<00:00, 112.20it/s]


Processed / All (fps 60): 249/14055


Processing: TotalCapture: 100%|██████████| 37/37 [00:00<00:00, 39.65it/s]


...

Processed / All (fps 120): 13588/14055


Processing: ACCAD: 100%|██████████| 252/252 [00:02<00:00, 101.98it/s]


Processed / All (fps 120): 13840/14055


Processing: MPI_HDM05: 100%|██████████| 215/215 [00:03<00:00, 55.32it/s]


Processed / All (fps 120): 14055/14055


The above code will extract poses from **AMASS** dataset, and put them under directory **"./pose_data"**

The source data from **HumanAct12** is already included in **"./pose_data"** in this repository. You need to **unzip** it right in this folder.

## Segment, Mirror and Relocate Motions

In [45]:
import codecs as cs
import pandas as pd
import numpy as np
from tqdm import tqdm
from os.path import join as pjoin

In [46]:
def swap_left_right(data):
    assert len(data.shape) == 3 and data.shape[-1] == 3
    data = data.copy()
    data[..., 0] *= -1
    right_chain = [2, 5, 8, 11, 14, 17, 19, 21]
    left_chain = [1, 4, 7, 10, 13, 16, 18, 20]
    left_hand_chain = [22, 23, 24, 34, 35, 36, 25, 26, 27, 31, 32, 33, 28, 29, 30]
    right_hand_chain = [43, 44, 45, 46, 47, 48, 40, 41, 42, 37, 38, 39, 49, 50, 51]
    tmp = data[:, right_chain]
    data[:, right_chain] = data[:, left_chain]
    data[:, left_chain] = tmp
    if data.shape[1] > 24:
        tmp = data[:, right_hand_chain]
        data[:, right_hand_chain] = data[:, left_hand_chain]
        data[:, left_hand_chain] = tmp
    return data

In [57]:
INFANT = True

if INFANT:
    index_path = './index_infant.csv'
    save_dir = './infant_joints'
else:
    index_path = './index.csv'
    save_dir = './joints'

if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    
index_file = pd.read_csv(index_path)
total_amount = index_file.shape[0]
fps = 15

In [58]:
for i in tqdm(range(total_amount)):
    source_path = index_file.loc[i]['source_path']
    new_name = index_file.loc[i]['new_name']
    data = np.load(source_path)
    start_frame = index_file.loc[i]['start_frame']
    end_frame = index_file.loc[i]['end_frame']
    if 'humanact12' not in source_path:
        if 'Eyes_Japan_Dataset' in source_path:
            data = data[3*fps:]
        if 'MPI_HDM05' in source_path:
            data = data[3*fps:]
        if 'TotalCapture' in source_path:
            data = data[1*fps:]
        if 'MPI_Limits' in source_path:
            data = data[1*fps:]
        if 'Transitions_mocap' in source_path:
            data = data[int(0.5*fps):]
        data = data[start_frame:end_frame]
        data[..., 0] *= -1
    
    data_m = swap_left_right(data)
#     save_path = pjoin(save_dir, )
    np.save(pjoin(save_dir, new_name), data)
    np.save(pjoin(save_dir, 'M'+new_name), data_m)

100%|██████████| 13/13 [00:00<00:00, 706.14it/s]
